This Notebook explains how to pass dataset between scripts

In [1]:
#import os
#os.makedirs('script_exp_dfload')

In [11]:
%%writefile script_exp_dfload/script.py
from azureml.core import Run,Dataset
import pandas as pd
import argparse as arg

parser = arg.ArgumentParser()
parser.add_argument('--ds',type=str,dest='dataset_name')
args = parser.parse_args()


run = Run.get_context()
ws = run.experiment.workspace
dataset = Dataset.get_by_name(ws,name=args.dataset_name)
print(dataset.take(3).to_pandas_dataframe())

Overwriting script_exp_dfload/script.py


In [12]:
from azureml.core import Workspace,Datastore,Dataset

In [13]:
ws = Workspace.from_config()
print(ws)

Workspace.create(name='mlws', subscription_id='d8f2241f-6180-41e3-a1de-149047238a9a', resource_group='mlrg')


In [14]:
#creating Environment as yaml file
from azureml.core import Environment,conda_dependencies,ScriptRunConfig
env = Environment('my_env')
packages = conda_dependencies.CondaDependencies.create(conda_packages=['pip'],
                                        pip_packages=['azureml-defaults','pandas']
                                        )
env.python.conda_dependencies = packages

script_config = ScriptRunConfig(source_directory="script_exp_dfload",script="script.py",arguments=['--ds','wine quality'],environment=env)

In [15]:
experiment_name = 'disp_df_data'
from azureml.core import Experiment
from azureml.widgets import RunDetails
exp = Experiment(workspace= ws,name = experiment_name)
run = exp.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'disp_df_data_1646834581_e688d68f',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2022-03-09T14:03:56.94377Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '4c5380d9-2c30-414d-9031-6ffaf3f05a15',
  'azureml.git.repository_uri': 'https://github.com/sureshgorakala/AzureML.git',
  'mlflow.source.git.repoURL': 'https://github.com/sureshgorakala/AzureML.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '7a446fba2a8cecd28e0c6fcc50e37d832587e26a',
  'mlflow.source.git.commit': '7a446fba2a8cecd28e0c6fcc50e37d832587e26a',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [{'dataset': {'id': '94ee7913-8030-4ccf-aacb-515aaf9a4236'}, 'consumptionDetails': {'type': 'Reference'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'script.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--ds', 'wine quality'],
  'sourceDirectoryDataStore': None,
  'framew